# Import Library

In [8]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Visuzliation Setting
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from matplotlib import colors
import seaborn as sns

---

# Wrangling Hypothesis
### 기본 전처리로 포함되어야 할 부분
1. Categorical values need one hot encoding
2. 

### 날짜
날짜를 year, month, day 로 처리하면 결국 int형 데이터가 되는데, 정량적으로 의미 없는 숫자가 악영향을 끼칠 수도 있지 않을까  
- datetime 형식으로 가져가기
    - 시계열 분석을 사용하는 것이 아니므로 제외한다
- year, month, day int형으로 가져가기
- 날짜와 관련있는 데이터를 다 넣어준 후(ex. 2015/09/15 노르웨이 기온 추가 후) 날짜 데이터 제거하기
- 날짜 데이터도 one hot encoding (2015년이다/아니다, 05월이다/아니다..)
> 1. year, month, day 유지한 dataframe
> 2. 날짜 데이터 제거한 dataframe
> 3. 날짜 데이터를 카테고리 변수화 하여 one hot encoding 한 dataframe
> 4. 분기별로 월을 분류하여 계절성을 부여한 dataframe

### 어종별 수
어종(P_NAME)에 따라서 수 차이가 있는데, 이를 수치화하여 새로운 데이터로 사용할 수 있다  
- P_NAME만 사용하기
- 어종별 수 세려서 새로운 컬럼 만들기
> 1. 기본 dataframe
> 2. 어종별 수를 고려한 dataframe

### 카테고리
어종 카테고리가 대분류, 중분류, 소분류 (CATEGORY_1, CATEGORY_2, P_NAME)으로 계츨 구조를 가지고 있다  
이를 어떻게 처리해야할까  
- 대분류에서 끊기
- 별개의 변수로 생각하여 소분류까지 포함하기
- 소분류만 사용하기
> 1. 기본 dataframe
> 2. CATEGORY_1 만 사용한 dataframe
> 3. P_NAME 만 사용한 dataframe

### 수입용도
P_IMPORT_TYPE 는 여러개의 수입 용도가 concate 되어 들어오므로 이를 분리하여 생각해야 한다  
- 여러개의 수입 용도를 가지는 품목의 경우 영향을 끼칠 수 있다고 생각
> 1. 기본 dataframe
> 2. P_IMPORT_TYPE의 수를 세린 컬럼을 추가한 dataframe

### 날씨
- 결측치 처리 방식 (fillna() 함수의 method 선택)
- 결측치 처리 시점 (raw_data 상태, 나라별 묶은 후, 최종 데이터)
- outlier 검출 방식 (정규화 후 끝부분 자르기, 나라별 최저/최고 기온 검색하여 처리)
- 해안가만 추출하기  
[pandas.DataFrame.fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)  
*method: 어떻게 NaN을 채울 것인가 결정  
limit: NaN 연속 몇개까지 채워줄 것인가 결정*  
> 1. fillna(method=ffill) 사용
> 2. fillna(method=bfill) 사용

> 결측치 처리 시점은 결측치에 대한 EDA를 더 진행한 후 생각하기

> 1. 기본
> 2. 정규화 후 끝부분을 잘라내는 방법으로 이상치 제거한 dataframe
> 3. 나라별로 이상치 예측하여 제거한 dataframe

> 1. 기본
> 2. 해안가 데이터 제외 제거한 dataframe

### 소비자물가지수 / 생활물가지수
- 전체 지수 평균 사용
- 품목 중 관련성 있는 지수 사용
- 둘다 사용

### 날씨 / 원유 / 소비자물가지수 / 생활물가지수 / 우리나라 날씨
- 사용
- 미사용

### Normalization of Numerical Values
temperature, wind, rain, temperature_kr, wind_kr, water_kr, 종가, etc 컬럼들의 분포 확인 후 normalization 진행
- Normalization 진행 여부
> 1. 기본
> 2. normalization 진행한 dataframe

### Data Filtering - Correlation
- 상관계수 계산해서 중복 변수 제거하기

### Data Filtering - Feature Selection (PCA...)
- 중요도가 높은 변수들만 선택하여 학습하기
- 중요도가 낮은 변수들은 제거하여 학습하기


### 전체
- 주별로 묶기
- 월별로 묶기
- 연별로 묶기
- 계절별로 묶기 (분기별)

<table style="border:1px solid gray">
    <thead style="border:1px solid gray">
        <tr>
            <th>index</th>
            <th>df명</th>
            <th>설명</th>
            <th>비교</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>0</td>
            <td>df0_base</td>
            <td>최소한의 전처리만 진행</td>
            <td>다른 dataframe 검증할 때 비교</td>
        </tr>
        <tr style="border:1px solid gray">
            <td colspan=4 style="text-align: center"><b>Date</b></td>
        </tr>
        <tr>
            <td>1</td>
            <td>df1_date</td>
            <td>날짜 데이터 포함</td>
            <td rowspan=3>df0_base, df1_date, df3_date 비교 후,<br>데이터 추가 후 df2_date와 비교</td>
        </tr>
        <tr>
            <td>2</td>
            <td>df2_date</td>
            <td>날짜 데이터 제거</td>
        </tr>
        <tr>
            <td>3</td>
            <td>df3_date</td>
            <td>날짜 데이터 one hot encoding</td>
        </tr>
        <tr style="border:1px solid gray">
            <td colspan=4 style="text-align: center"><b>Adding columns</b></td>
        </tr>
        <tr>
            <td>4</td>
            <td>df4_name</td>
            <td>어종별 수 컬럼 추가</td>
            <td rowspan=2>df0_base<br>df4_name<br>df5_import</td>
        </tr>
        <tr>
            <td>5</td>
            <td>df5_import</td>
            <td>수입용도 수 컬럼 추가</td>
        </tr>
        <tr style="border:1px solid gray">
            <td colspan=4 style="text-align: center"><b>Dropping Columns</b></td>
        </tr>
        <tr>
            <td>6</td>
            <td>df6_caegory</td>
            <td>CATEGORY_2, P_NAME 컬럼 제거</td>
            <td rowspan=2>df0_base<br>df6_category<br>df7_category</td>
        </tr>
        <tr>
            <td>7</td>
            <td>df7_category</td>
            <td>CATEGORY_1, CATEGORY_2 컬럼 제거</td>
        </tr>
        <tr style="border:1px solid gray">
            <td colspan=4 style="text-align: center"><b>Adding Data</b></td>
        </tr>
        <tr>
            <td>8</td>
            <td>df8_add</td>
            <td>제조국 날씨, 한국 날씨, 원유 종가, 염도, 소비자물가지수 data 추가</td>
            <td>feature 중요도 파악<br>상관관계 파악</td>
        </tr>
        <tr style="border:1px solid gray">
            <td colspan=4 style="text-align: center"><b>Normalization</b></td>
        </tr>
        <tr>
            <td>9</td>
            <td>df9_add</td>
            <td>numerical feature 에 대한 정규화 진행</td>
            <td>df0_base<br>feature 중요도 파악<br>상관관계 파악</td>
        </tr>
        <tr style="border:1px solid gray">
            <td colspan=4 style="text-align: center"><b>Filling Missing Values</b></td>
        </tr>
        <tr>
            <td>10</td>
            <td>df10_fillna</td>
            <td>fillna(method=ffill)로 날씨 결측치 처리</td>
            <td rowspan=2>df0_base<br>df10_fillna<br>df11_fillna</td>
        </tr>
        <tr>
            <td>11</td>
            <td>df11_fillna</td>
            <td>fillna(method=bfill)로 날씨 결측치 처리</td>
        </tr>
        <tr style="border:1px solid gray">
            <td colspan=4 style="text-align: center"><b>Processing Outliers</b></td>
        </tr>
        <tr>
            <td>12</td>
            <td>df12_outlier</td>
            <td>전체 날씨 데이터 정규화 후 outlier 일괄 제거</td>
            <td rowspan=2>df0_base<br>df12_outlier<br>df13_outlier</td>
        </tr>
        <tr>
            <td>13</td>
            <td>df13_outlier</td>
            <td>나라별로 최저/최고 기온 파악 후 outlier 개별 제거</td>
        </tr>
    </tbody>
</table>

---

In [9]:
root = os.path.join(os.getcwd(), 'DATA')

# Import Data

In [19]:
df_train = pd.read_excel(os.path.join(root, 'train.xlsx'))
df_weather_code = pd.read_excel(os.path.join(root, 'raw_weather_code.xlsx'), header=None, index_col=0)
weather_list = [pd.read_csv(os.path.join(root, 'raw_weather_20151228_20161227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20161228_20171227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20171228_20181227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20181228_20191227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20191228_20201227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20201228_20210818.csv'), encoding='euc-kr')]
df_oil = pd.read_csv(os.path.join(root, 'preprocessed_oil.csv'))
df_weather_kr = pd.read_csv(os.path.join(root, 'preprocessed_weather_korea.csv'))
df_cpi1 = pd.read_csv(os.path.join(root, 'preprocessed_cpi_region.csv'))
df_cpi2 = pd.read_csv(os.path.join(root, 'preprocessed_cpi_purpose.csv'))

제조국 날씨의 경우 결측치 처리 과정이 필요하기 때문에 raw data를 가져온다

## df0_base
최소한의 전처리만 진행

In [20]:
df0_base = df_train.copy()

In [21]:
# One hot encoding
import_type_list = set()
for tmp in df0_base.P_IMPORT_TYPE.unique():
    for a in tmp.split(','):
        import_type_list.add(a)
for name in import_type_list:
    df0_base[name] = 0
    df0_base.loc[df_train['P_IMPORT_TYPE'].str.contains(name, regex=False), name] = 1

In [22]:
# One hot encodiing
one_hot = ['CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1', 'CATEGORY_2', 'P_NAME']
df0_base = pd.get_dummies(df_train, columns=one_hot)

In [23]:
# Drop columns
drop = ['P_TYPE', 'P_IMPORT_TYPE']
df0_base.drop(columns = drop, inplace=True)

In [38]:
print(df0_base.columns)

Index(['P_PRICE', 'CTRY_1_가나', 'CTRY_1_감비아', 'CTRY_1_그리스', 'CTRY_1_그린란드',
       'CTRY_1_기니', 'CTRY_1_기니비사우', 'CTRY_1_나미비아', 'CTRY_1_남아프리카 공화국',
       'CTRY_1_네덜란드',
       ...
       'P_NAME_황돔', 'P_NAME_황새치', 'P_NAME_황적퉁돔', 'P_NAME_회초리꼬리민태',
       'P_NAME_흑기흉상어', 'P_NAME_흑밀복', 'P_NAME_흑점샛돔알', 'P_NAME_흑점줄전갱이',
       'P_NAME_흰꼴뚜기', 'P_NAME_흰다리새우'],
      dtype='object', length=587)


## df1_date
날짜 데이터 int형으로 변환

In [25]:
df1_date = df0_base.copy()

In [26]:
# 날짜변수 추가
df1_date['year'] = df1_date['REG_DATE'].dt.year
df1_date['month'] = df1_date['REG_DATE'].dt.month
df1_date['day'] = df1_date['REG_DATE'].dt.day

In [30]:
df1_date.drop(columns = ['REG_DATE'], inplace=True)

In [39]:
print(df1_date.columns)

Index(['P_PRICE', 'CTRY_1_가나', 'CTRY_1_감비아', 'CTRY_1_그리스', 'CTRY_1_그린란드',
       'CTRY_1_기니', 'CTRY_1_기니비사우', 'CTRY_1_나미비아', 'CTRY_1_남아프리카 공화국',
       'CTRY_1_네덜란드',
       ...
       'P_NAME_회초리꼬리민태', 'P_NAME_흑기흉상어', 'P_NAME_흑밀복', 'P_NAME_흑점샛돔알',
       'P_NAME_흑점줄전갱이', 'P_NAME_흰꼴뚜기', 'P_NAME_흰다리새우', 'year', 'month', 'day'],
      dtype='object', length=590)


## df3_date
날짜 데이터 one hot encoding

In [32]:
df3_date = df1_date.copy()

In [41]:
# One hot encodiing
df3_date = pd.get_dummies(df3_date, columns=['year', 'month', 'day'])